In [7]:
import numpy as np
import psycopg2
import cv2
import csv
import PIL
import os
import random
from PIL import Image
import argparse
import pickle
from sklearn.utils import shuffle

parser = argparse.ArgumentParser(description='separating images into three categories')
parser.add_argument('-s', '--schema', default='gare_st_lazare', 
                    help='schema for the station')
parser.add_argument('-i', '--imagelist', default='images_225', 
                    help='image list table name')
parser.add_argument('-tp', '--trainpath', default='../data/train_lazare', 
                    help='training data path')
parser.add_argument('-vp', '--valpath', default='../data/val_lazare', 
                    help='validation data path')
parser.add_argument('-ep', '--testpath', default='../data/test_lazare', 
                    help='testing data path')
parser.add_argument('-p', '--percentagetrain', default=0.8, type=float,
                    help='train data percentage in all data')

args = parser.parse_args()

schema = args.schema
image_list_table = args.imagelist

per_train = args.percentagetrain
per_val   = (1 - per_train)/2
per_test  = 1 - per_train - per_test

train_path = args.trainpath
val_path   = args.valpath
test_path  = args.testpath

usage: ipykernel_launcher.py [-h] [-s SCHEMA] [-i IMAGELIST] [-tp TRAINPATH]
                             [-vp VALPATH] [-ep TESTPATH] [-p PERCENTAGETRAIN]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-51ebd586-5a86-41df-b348-e79f26c8d236.json


SystemExit: 2

In [ ]:

data_train = np.array([], dtype=np.uint8).reshape(0, 150528)
label_train = np.array([], dtype=np.uint8)

data_val = np.array([], dtype=np.uint8).reshape(0, 150528)
label_val = np.array([], dtype=np.uint8)

data_test = np.array([], dtype=np.uint8).reshape(0, 150528)
label_test = np.array([], dtype=np.uint8)

conn_string = "host='localhost' dbname='indoor_position' user='postgres' password='tiancai' port='5432'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()
query = '''select distinct(category) from %s.%s''' % (schema, image_list_table)
cur.execute(query)
categories = cur.fetchall()
conn.commit()
zone_id_lookup = {}

f = csv.writer(open('category.csv','w+'))

for idx, item in enumerate(categories):
    query = '''select * from %s.%s where category = '%s' and orientation != pano; ''' % (schema, image_list_table, item[0])

    cur.execute(query)
    conn.commit()
    images = cur.fetchall()
    category = item[0]
    
    f.writerow([category, idx])

    # print(images)
    data_category = np.array([cv2.resize(cv2.cvtColor(np.array(Image.open(fname[2])), cv2.COLOR_BGR2RGB), dsize=(224, 224), interpolation=cv2.INTER_CUBIC) for fname in images])	
    data_tag = [fname[0] for fname in images]
    
    data_category, data_tag = shuffle(data_category, data_tag, random_state=0)
    
    # data_category = np.load(file_np).astype(np.int8)
    total_cnt = len(data_category)
    num_train = int(per_train * total_cnt)
    num_val   = int(per_val * total_cnt)
    num_test  = int(per_test * total_cnt)
    print (num_train, num_val, num_test, total_cnt)
    # generate split

    train_category = data_category[:num_train]
    val_category = data_category[num_train:num_train + num_val]
    test_category = data_category[num_train + num_val: ]
    
    train_tag = data_tag[:num_train]
    val_tag  = data_tag[num_train:num_train + num_val]
    test_tag = data_tag[num_train + num_val: ]
    
    if not os.path.exists(os.path.join(train_path, item[0])):
        os.makedirs(os.path.join(train_path, item[0]))
    if not os.path.exists(os.path.join(val_path, item[0])):
        os.makedirs(os.path.join(val_path, item[0]))
    if not os.path.exists(os.path.join(test_path, item[0])):
        os.makedirs(os.path.join(test_path, item[0]))
                    
    for id, image in enumerate(train_category):
        cv2.imwrite(os.path.join(train_path, item[0], '%s.png' % train_tag[id]), image)
    for id, image in enumerate(val_category):
        cv2.imwrite(os.path.join(val_path, item[0], '%s.png' % val_tag[id]), image)
    for id, image in enumerate(test_category):
        cv2.imwrite(os.path.join(test_path, item[0], '%s.png' % test_tag[id]), image)

    print(data_train.shape, train_category.shape)
    print('num_train=%d num_val=%d num_test=%d' % (data_train.shape[0], data_val.shape[0], data_test.shape[0]))
                    